In [1]:
import numpy as np
import json
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.util import ngrams
from stop_words import get_stop_words
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nqabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_original = pd.read_json('tesla_tweets.json', encoding='utf-8')
df = df_original.copy()

In [3]:
df = df[['timestamp', 'text', 'username']]

In [4]:
df.set_index(pd.to_datetime(df['timestamp']), inplace=True)
df.drop(columns='timestamp', inplace=True)
df.head()

,text,username
timestamp,,
2011-07-20 19:55:32,Tesla wins $100 million Toyota supply deal | h...,Bloomberg
2011-05-25 01:22:17,Tesla CEO: No interest in selling electric car...,Bloomberg
2011-07-20 19:55:32,Tesla wins $100 million Toyota supply deal | h...,Bloomberg
2011-05-25 01:22:17,Tesla CEO: No interest in selling electric car...,Bloomberg
2012-05-07 23:45:12,Toyota to sell electric RAV4 with Tesla Motor ...,Bloomberg


In [5]:
df.drop_duplicates(subset=['text'], keep=False, inplace=True)

In [36]:
def generate_clean_ngrams(tweet_text, n):
    
    # create list of stop words from two different libraries
    stop_words = list(stopwords.words('english')) + list(get_stop_words('en'))
    
    # convert tweet to lower case and split into words
    word_list = tweet_text.lower().split(' ')
    
    # remove (most) links
    word_list = [word for word in word_list if 'http' not in word]
    
    # identify numbers to later remove words with numbers
    pattern = '[0-9]'
    
    # replace special characters with a space
    word_list = re.sub(r'[^a-zA-Z0-9\s]', ' ', word_list)
    
    # replace numbers with a space
    word_list = re.sub(pattern, ' ', word_list)
    

    tokens = [token for token in word_list if token != '' 
              and token not in stop_words]
    
    output = list(ngrams(tokens, n))
    
    return output

In [38]:
def generate_clean_ngrams(tweet_text, n):
    
    # create list of stop words from two different libraries
    stop_words = list(stopwords.words('english')) +\
                 list(get_stop_words('en'))
    
    # identify numbers to later remove words with numbers
    pattern = '[0-9]'
    
    # convert words to lower case
    tweet_text = tweet_text.lower()
    
    # replace special characters with a space
    tweet_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', tweet_text)
    
    # replace numbers with string 'http' to later remove both links 
    # and words with numbers in one line
    tweet_text = re.sub(pattern, 'http', tweet_text)
    
    tokens = [token for token in tweet_text.split(' ') 
              if token != '' 
              and token not in stop_words 
              and 'http' not in token]
    
    output = list(ngrams(tokens, n))
    
    return output

In [39]:
df['ngrams'] = df['text'].map(lambda x: generate_clean_ngrams(x, n=2))

In [40]:
df.head()

,text,username,ngrams
timestamp,,,
2014-05-08 14:00:18,"Tech shares slide, Tesla drops:\nhttp://bloom....",Bloomberg,"[(tech, shares), (shares, slide), (slide, tesl..."
2014-05-07 22:00:23,"Diminished credits, supplies crimp Tesla's fir...",Bloomberg,"[(diminished, credits), (credits, supplies), (..."
2014-05-07 20:07:23,EARNINGS: Tesla Q1 adjusted EPS of 12 cents be...,Bloomberg,"[(earnings, tesla), (tesla, adjusted), (adjust..."
2014-04-25 14:00:45,"Elon Musk, Tesla’s 42-year-old CEO, received $...",Bloomberg,"[(elon, musk), (musk, tesla), (tesla, year), (..."
2014-04-24 23:03:13,RT @BloombergTV Tesla hands keys to first Chin...,Bloomberg,"[(rt, bloombergtv), (bloombergtv, tesla), (tes..."


In [51]:
df.text[999]

'Tesla plans up to $1.15 billion capital raise to cut risk http://bloom.bg/2noEGIM\xa0pic.twitter.com/64AHh3i5fm'

In [56]:
list(df.ngrams[999])[:-4]

[('tesla', 'plans'),
 ('plans', 'billion'),
 ('billion', 'capital'),
 ('capital', 'raise'),
 ('raise', 'cut'),
 ('cut', 'risk')]

In [28]:
dummies = pd.get_dummies(pd.DataFrame(df2['ngrams'].tolist()).stack()).sum(level=0)

In [ ]:
dummies.columns = list(map(lambda x: ', '.join(x), list(dummies.columns)))

In [32]:
drop_columns = ['\n, anniversary',
 '\n, ecb',
 '\n, joe',
 '\n, kind',
 '\n, mile',
 '\n, price',
 '\n, pure',
 '\n, venezuela',
 '\n\n, mr',
 '\n\nafter, increasing',
 '\n\nback, earth',
 '\n\nbut, many',
 '\n\nfix, repeat',
 '\n\nfollow, along',
 '\n\nhas, really',
 '\n\nhere, latest',
 '\n\nhere, need',
 '\n\nstruggle, make',
 '\n\nthe, man',
 '\n\nthe, tesla',
 '\n\nwe, asked',
 '\n\nwhat, tesla',
 '\nelon, musk',
 '\nget, line',
 '\ntesla, motors',
 '\nthe, model',
 '\nwait, times']
dummies = dummies.drop(columns=drop_columns)

In [33]:
dummies.index = df2.index

In [ ]:
dummies.head(5)

In [35]:
df2.head(5)

,text,username,sentiment_score,sentiment_pos,sentiment_neg,ngrams
timestamp,,,,,,
2014-05-08 14:00:18,"Tech shares slide, Tesla drops:\nhttp://bloom....",Bloomberg,0.000000,0,0,"[(tech, shares), (shares, slide), (slide, tesl..."
2014-05-07 22:00:23,"Diminished credits, supplies crimp Tesla's fir...",Bloomberg,0.047222,1,0,"[(diminished, credits), (credits, supplies), (..."
2014-05-07 20:07:23,EARNINGS: Tesla Q1 adjusted EPS of 12 cents be...,Bloomberg,0.000000,0,0,"[(earnings, tesla), (tesla, adjusted), (adjust..."
2014-04-25 14:00:45,"Elon Musk, Tesla’s 42-year-old CEO, received $...",Bloomberg,-0.077778,0,1,"[(elon, musk), (musk, tesla), (tesla, year), (..."
2014-04-24 23:03:13,RT @BloombergTV Tesla hands keys to first Chin...,Bloomberg,0.250000,1,0,"[(rt, bloombergtv), (bloombergtv, tesla), (tes..."


In [36]:
df_tweets = pd.concat([df2, dummies], axis=1)

In [37]:
df_tweets.head(5)

,text,username,sentiment_score,sentiment_pos,sentiment_neg,ngrams,"aapl, tslapic","abandon, plan","abandon, proposed","abandoned, plan",...,"yuan, steady","yueting, aims","zachary, kirkhorn","zero, elon","zone, crashed","zooms, past","zoox, four",", prefer","here, happened","tesla, unveil"
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-05-08 14:00:18,"Tech shares slide, Tesla drops:\nhttp://bloom....",Bloomberg,0.000000,0,0,"[(tech, shares), (shares, slide), (slide, tesl...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-05-07 22:00:23,"Diminished credits, supplies crimp Tesla's fir...",Bloomberg,0.047222,1,0,"[(diminished, credits), (credits, supplies), (...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-05-07 20:07:23,EARNINGS: Tesla Q1 adjusted EPS of 12 cents be...,Bloomberg,0.000000,0,0,"[(earnings, tesla), (tesla, adjusted), (adjust...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-04-25 14:00:45,"Elon Musk, Tesla’s 42-year-old CEO, received $...",Bloomberg,-0.077778,0,1,"[(elon, musk), (musk, tesla), (tesla, year), (...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-04-24 23:03:13,RT @BloombergTV Tesla hands keys to first Chin...,Bloomberg,0.250000,1,0,"[(rt, bloombergtv), (bloombergtv, tesla), (tes...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
df3 = df_tweets.groupby(pd.Grouper(freq='D')).mean()

In [39]:
df3 = df_tweets.groupby(pd.Grouper(freq='D')).mean()
df3.sort_index(inplace=True)
df3.head()

,sentiment_score,sentiment_pos,sentiment_neg,"aapl, tslapic","abandon, plan","abandon, proposed","abandoned, plan","abandoned, short","abandoning, tesla","abandons, buyout",...,"yuan, steady","yueting, aims","zachary, kirkhorn","zero, elon","zone, crashed","zooms, past","zoox, four",", prefer","here, happened","tesla, unveil"
timestamp,,,,,,,,,,,,,,,,,,,,,
2013-03-05,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df3 = df3.shift(1)

In [41]:
df3 = df3.shift(1)
df3.head()

,sentiment_score,sentiment_pos,sentiment_neg,"aapl, tslapic","abandon, plan","abandon, proposed","abandoned, plan","abandoned, short","abandoning, tesla","abandons, buyout",...,"yuan, steady","yueting, aims","zachary, kirkhorn","zero, elon","zone, crashed","zooms, past","zoox, four",", prefer","here, happened","tesla, unveil"
timestamp,,,,,,,,,,,,,,,,,,,,,
2013-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-06,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df = df1.join(df3, sort=True)

In [43]:
df.dropna(inplace=True)
df.drop(columns=['Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'sentiment_pos',
 'sentiment_neg'], inplace=True)

In [44]:
df.sort_index(ascending=False, inplace=True)

In [45]:
df.head(5)

,POC,UP_3%,DOWN_3%,sentiment_score,"aapl, tslapic","abandon, plan","abandon, proposed","abandoned, plan","abandoned, short","abandoning, tesla",...,"yuan, steady","yueting, aims","zachary, kirkhorn","zero, elon","zone, crashed","zooms, past","zoox, four",", prefer","here, happened","tesla, unveil"
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-29,0.009,0,0,0.046111,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-28,-0.003,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-27,0.030,1,0,-0.250000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-24,-0.010,0,0,0.115476,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-23,0.014,0,0,0.093466,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
df['target'] = ''
for i in df.index:
    
    if df['UP_3%'][i] == 1:
        df['target'][i] = 1
    elif df['DOWN_3%'][i] == 1:
        df['target'][i] = 2
    else:
        df['target'][i] = 0

C:\Users\nqabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\nqabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\nqabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [47]:
df['target'].value_counts()

0    492
1     55
2     51
Name: target, dtype: int64

In [48]:
df['target'] = df['target'].astype(int)

In [49]:
df['UP_3%'].value_counts()

0    543
1     55
Name: UP_3%, dtype: int64

In [50]:
df['DOWN_3%'].value_counts()

0    547
1     51
Name: DOWN_3%, dtype: int64

In [51]:
df_model = df[df['target'] != 0]
df_model.shape

(106, 14150)

In [ ]:
import nltk
from nltk import word_tokenize, regexp_tokenize

text_A = [word.lower() for 
                word in word_tokenize(texts[0])]
text_A

In [ ]:
import string
string.punctuation

text_A = [word for word in text_A
         if word not in string.punctuation]
text_A

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

text_A = [word for word in text_A 
          if text_A not in stopwords.words('english')]
text_A

In [ ]:
from nltk.stem import *

p_stemmer = PorterStemmer()

text_A_pstem = [p_stemmer.stem(word) for word in text_A]
text_A_pstem

In [ ]:
from nltk import pos_tag

text_A_pos = pos_tag(text_A)
text_A_pos[2]

In [58]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from stop_words import get_stop_words
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nqabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [128]:
tweet = df.text[3]
tweet

'Elon Musk, Tesla’s 42-year-old CEO, received $69,989 in 2013, down from $78.2 million a year earlier: http://bloom.bg/1nMG7XY\xa0'

In [129]:
urls = re.findall('http.+?\s', tweet)
urls

['http://bloom.bg/1nMG7XY\xa0']

In [130]:
tweet = ' '.join([word for word in tweet.split(' ') if word not in urls])
tweet

'Elon Musk, Tesla’s 42-year-old CEO, received $69,989 in 2013, down from $78.2 million a year earlier:'

In [115]:
df.text

timestamp
2014-05-08 14:00:18    Tech shares slide, Tesla drops:\nhttp://bloom....
2014-05-07 22:00:23    Diminished credits, supplies crimp Tesla's fir...
2014-05-07 20:07:23    EARNINGS: Tesla Q1 adjusted EPS of 12 cents be...
2014-04-25 14:00:45    Elon Musk, Tesla’s 42-year-old CEO, received $...
2014-04-24 23:03:13    RT @BloombergTV Tesla hands keys to first Chin...
                                             ...                        
2018-08-08 03:40:29    Tesla buyout would add to year of pain for sho...
2018-08-08 03:03:06    To take Tesla private, Elon Musk would have to...
2018-08-08 02:28:43    - Asia stocks broadly higher, but China shares...
2018-08-08 01:42:38    Elon Musk has a plan to quiet Tesla critics fo...
2018-08-08 00:45:07    Regulators could examine the facts behind Elon...
Name: text, Length: 3224, dtype: object

In [114]:
urls = re.findall('http.+?\s', tweet)
tweet = [' '.join([word for word in tweet.split(' ') if word not in urls])]
pattern = ("([a-zA-Z]+(?:'[a-z]+)?)")
cv = CountVectorizer(token_pattern=pattern, 
                     stop_words=nltk.corpus.stopwords.words('english'),
                     ngram_range=(2,2))

cv_texts = cv.fit_transform()
cv_df = pd.DataFrame(cv_texts.toarray())
cv_df.columns = cv.vocabulary_
cv_df

,glencore negotiating,negotiating long,long term,term contract,contract ship,ship cobalt,cobalt tesla,tesla new,new electric,electric vehicle,vehicle factory,factory shanghai
0,1,1,1,1,1,1,1,1,1,1,1,1


In [103]:
tweet = df.text[2345]
urls = re.findall('http.+?\s', tweet)
tweet = ' '.join([word for word in tweet.split(' ') if word not in urls])
tweet

'Glencore is negotiating a long-term contract to ship cobalt to Tesla’s new electric-vehicle factory in Shanghai'

In [ ]:
def generate_clean_ngrams(tweet_text, n):
    
    # create list of stop words from two different libraries
    stop_words = list(stopwords.words('english')) +\
                 list(get_stop_words('en'))
    
    # identify numbers to later remove words with numbers
    pattern = '[0-9]'
    
    # convert words to lower case
    tweet_text = tweet_text.lower()
    
    # replace special characters with a space
    tweet_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', tweet_text)
    
    # replace numbers with string 'http' to later remove both links 
    # and words with numbers in one line
    tweet_text = re.sub(pattern, 'http', tweet_text)
    
    tokens = [token for token in tweet_text.split(' ') 
              if token != '' 
              and token not in stop_words 
              and 'http' not in token]
    
    output = list(ngrams(tokens, n))
    
    return output